In [1]:
########################################################################################################################
# This script extracts the needed potential predictors as encounter-level features
# Step 1. Identify the needed potential predictors from the data dictionary
# Step 2. Identify the needed rows (i.e., encounters to be considered)
# Step 3. Retain only variables with adequate records
# Step 4. Split into training and test set using the target data created in script P02_Stratified_Partitioning
########################################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import numpy as np
import os
import pandas as pd
import warnings
from ast import literal_eval
from itertools import product, chain
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
####################################################################################################################
# USER_SPECIFIC SETTING
# ENC_PATH: Path of the encounter-level dataset 
# TARGET_DIR_PATH: Path of the directory storing the target datasets created in P01_Subject_Inclusion.ipynb
# OUT_DIR_PATH: Path of the directory of the output feature datasets
####################################################################################################################
ENC_PATH: str = '../00_Data/01_Cleaned_Data/Encounter_full_v2.parquet'
TARGET_DIR_PATH: str = '../00_Data/02_Processed_Data/Targets/'
OUT_FEAT_DIR_PATH: str = '../00_Data/02_Processed_Data/Features/'

In [ ]:
########################################################################################################################
# USER_SPECIFIC SETTING
# NAN_THRESHOLD: A float p between 0 and 1 such that variables with less than (p * 100)% records will not be included
########################################################################################################################
NAN_THRESHOLD: float = 0.35

In [ ]:
########################################################################################################################
# USER-SPECIFIC SETTING
# Cs: Different numbers of feature encounteres to be included
# Ds: Different maximum widths of the look-back window in days
########################################################################################################################
Cs : list[int] = [1, 2, 3, 4]
Ds : list[int] = [60, 120, 180]

In [ ]:
####################################################################################################################
# Load the encounter-level dataset
####################################################################################################################
df_enc: pd.DataFrame = pd.read_parquet(ENC_PATH)
df_enc['EncDate'] = pd.to_datetime(df_enc['EncDate'])
unneeded_cols: list[str] = ['AgeInDays', 'AgeInWeeks', 'AgeInMonths']
df_enc.drop(columns=unneeded_cols, inplace=True, errors='ignore')

In [4]:
########################################################################################################################
# Identify the needed potential predictors and store them as the keys of a dictionary
########################################################################################################################
idx_cols: list[str] = ['PatientDurableKey', 'EncounterKey', 'EncDate']
enc_cols: list[str] = [c for c in df_enc.columns if c not in idx_cols]
print(f'Encounter-level variables to be extracted: {len(enc_cols)}')

Encounter-level variables to be extracted: 137


In [ ]:
########################################################################################################################
# Loop over the experiment configurations Cs and Ds
########################################################################################################################
for exp_idx, (C, D) in enumerate(product(Cs, Ds), 1):
    log_head: str = f'[{exp_idx}. C={C}; D={D}] '
    
    if C == 1 and Ds.index(D) > 0:      # When C=1, all D values are the same
        continue

    ####################################################################################################################
    # Load the target file created in P01_Subject_Inclusion
    ####################################################################################################################
    y_path: str = os.path.join(TARGET_DIR_PATH, f'{C}_encounters_{D}_days_v1.csv')
    df_y: pd.DataFrame = pd.read_csv(y_path)

    ####################################################################################################################
    # Identify the encounters to be included
    ####################################################################################################################
    df_y['FeatureEncKeys'] = df_y['FeatureEncKeys'].apply(lambda x: literal_eval(x))
    y_encs: list[list[int]] = list(chain.from_iterable(df_y['FeatureEncKeys'].values))
    assert len(y_encs) == len(np.unique(y_encs))                    # Encounters extracted are unique
    assert len(y_encs) == df_y['PatientDurableKey'].nunique() * C   # # Encounters == # Patients * C
    print(f'{log_head}Extracting {len(y_encs)} encounters from the encounter-level dataset.')
    
    ####################################################################################################################
    # Prepare an overall DataFrame as output
    ####################################################################################################################
    df_out: pd.DataFrame = pd.DataFrame(None)

    ####################################################################################################################
    # Loop over the encounter-level data files
    ####################################################################################################################
    for col_idx, col in enumerate(enc_cols, 1):
        log_head_sub: str = f'{log_head}<{col_idx}. {col}> '

        ################################################################################################################
        # Load the parquet file for the encounter-level variable 
        ################################################################################################################
        df_cur: pd.DataFrame = df_enc[['PatientDurableKey', 'EncDate', 'EncounterKey', col]]
        df_cur = df_cur[df_cur['EncounterKey'].isin(y_encs)]
        assert df_cur.shape[0] == len(y_encs)

        ################################################################################################################
        # Retain only columns with at least (1 - NAN_THRESHOLD)% data
        ################################################################################################################
        N: int = df_cur.shape[0]
        if df_cur[col].isna().sum() >= N * NAN_THRESHOLD:
            print(f'{log_head_sub}Skipped because >= {int(NAN_THRESHOLD*100)}% missing values')
            continue

        ################################################################################################################
        # Merge df_cur with df_out
        ################################################################################################################
        df_out = df_cur.copy(deep=True) if df_out.empty else pd.merge(left=df_out, right=df_cur, on=idx_cols, how='left')
        print(f'{log_head_sub}Merged.')

    ####################################################################################################################
    # Sanity check
    ####################################################################################################################
    if df_out.empty:
        raise ValueError(f"{log_head}No encounter-level features survived the specified NAN_THRESHOLD")
    
    ####################################################################################################################
    # Load the partitions of y separately
    ####################################################################################################################
    for partition in ['train', 'test']:
        y_part_path: str = y_path.replace('_v1.csv', f'_{partition}_v2.parquet')
        y_part: pd.DataFrame = pd.read_parquet(y_part_path)['PatientDurableKey'].to_list()
        df_out_part: pd.DataFrame = df_out[df_out['PatientDurableKey'].isin(y_part)]
        df_out_part = df_out_part.sort_values(by=['PatientDurableKey', 'EncDate', 'EncounterKey'], ascending=[True, True, True])
        
        ################################################################################################################
        # Save the patient-level feature dataset
        ################################################################################################################
        out_dir_path_s: str = os.path.join(OUT_FEAT_DIR_PATH, f'{C}_encounters_{D}_days/')
        os.makedirs(out_dir_path_s, exist_ok=True)
        out_file_path: str = f'{out_dir_path_s}X_Encounter_{partition}_v1.parquet'
        df_out_part.to_parquet(out_file_path)
        print(f'{log_head}({partition}) Encounter-level dataset saved with dimension = {df_out_part.shape}')
    print('-'*120)